In [2]:
import os
import math
import torch
import random
import shutil
import pickle
import pprint
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import models, transforms
from transformers import ViTFeatureExtractor, ViTModel, ViTImageProcessor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
warnings.filterwarnings("ignore")

/Users/xiaoyanli/opt/miniconda3/envs/cancer_immune/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Four models, Resnet152, Alexnet, VGG16, GoogleNet, ViT Classification

In [3]:
data_frame = pd.read_csv('oasis_cross-sectional.csv')
data_frame.drop('Delay', axis=1, inplace=True)
data_frame = data_frame.dropna()

with open('patients_codes_set.pkl', 'rb') as f:
    patients_codes_set = pickle.load(f)

print("Set loaded from patients_codes_set.pkl:", patients_codes_set)

Set loaded from patients_codes_set.pkl: {'0123', '0304', '0042', '0267', '0003', '0273', '0247', '0298', '0205', '0411', '0243', '0158', '0300', '0226', '0441', '0451', '0115', '0164', '0066', '0335', '0179', '0351', '0039', '0286', '0233', '0380', '0352', '0142', '0143', '0082', '0307', '0022', '0238', '0308', '0440', '0374', '0432', '0161', '0329', '0453', '0287', '0263', '0290', '0120', '0094', '0288', '0021', '0447', '0272', '0400', '0418', '0315', '0454', '0210', '0390', '0023', '0402', '0016'}


In [4]:
classification_dataframe = pd.DataFrame(columns=['Type', 'model', 'mode', 'Accuracy', 'Precision', 'Recall', 'True positive', 'Sensitivity', 'Specificity', 'F1 Score'])

lr = 1e-6
epochs = 100
num = 400
train_folder = './train_data'
# test_folder = './test_data'
test_folder = './train_data'

if not os.path.exists('./comparison'+'/models'):
    os.makedirs('./comparison'+'/models')

if not os.path.exists('./comparison'+'/results'):
    os.makedirs('./comparison'+'/results')

if not os.path.exists('./comparison'+'/test'):
    os.makedirs('./comparison'+'/test')

##resnet152-Classification

In [4]:
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
resnet152 = models.resnet152(pretrained=True)
model_name = 'resnet152'
# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel, self).__init__()
        self.vit_model = vit_model
        self.resnet152 = models.resnet152(pretrained=True)
        # for param in self.resnet152.parameters():
        #     param.requires_grad = False
        self.linear_layer1 = nn.Linear(1000, 100)
        # self.linear_layer1.weight.data.normal_(mean=0.0, std=0.02)
        self.linear_layer2 = nn.Linear(100, 4)
        # self.linear_layer2.weight.data.normal_(mean=0.0, std=0.02)
        self.relu = nn.ReLU()
        self.out_1 = nn.Linear(9, 4)
        # self.out_1.weight.data.normal_(mean=0.0, std=0.02)
        # self.out_2 = nn.Linear(5, 4)
        # self.out_2.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, inputs, x):
        # Preprocess the input tensor
        inputs = inputs.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=inputs, return_tensors="pt")
        outputs = self.resnet152(inputs['pixel_values'])
        outputs = self.linear_layer1(outputs)
        outputs = self.relu(outputs)
        final_output = self.linear_layer2(outputs)
        final_output = torch.cat((final_output, x), 1)
        final_output = self.relu(final_output)
        final_output = self.out_1(final_output)
        # final_output = self.relu(final_output)
        # final_output = self.out_2(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel(vit_model).to(device)
# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

Device: cpu


In [11]:
folders = os.listdir(train_folder)
folders = [folder for folder in folders if os.path.isdir(train_folder + '/'+folder)]
print("Folders:", folders)

Folders: ['VeryMildDemented', 'ModerateDemented', 'MildDemented', 'NonDemented']


In [12]:
index = []
for i in range(len(folders)):
    folder = folders[i]
    if i == 0:
        y = 1
    elif i == 1:
        y = 3
    elif i == 2:
        y = 2
    else:
        y = 0
    for file in os.listdir(train_folder + '/' + folder):
        if file.endswith('.npy'):
            index.append([train_folder + '/' + folder + '/' + file, y])
# print(index)
# temp = []
# for ele in index:
#     if ele[0][-14:-10] in patients_codes_set:
#         temp.append([ele[0], ele[1]])
# index = temp

In [7]:
# resnet152 model training with MR1_1
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Epoch 0 loss: 138.31804251670837
Model saved to comparison/models/resnet152_MR1_1_class.pth


In [8]:
# adding resnet152 MR1_1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder + '/' + folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            data = data/255
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.eval()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  resnet152  MR1_1  0.309192   0.319658  1.045956   

   True positive  Sensitivity  Specificity  F1 Score  
0            111     0.319658      0.77327  0.489667  


In [9]:
# resnet152 model training with MR1_2

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 160.98243534564972
Model saved to comparison/models/resnet152_MR1_2_class.pth


In [10]:
# adding resnet152 MR1_2 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  resnet152  MR1_2   0.24234   0.256071  1.058929   

   True positive  Sensitivity  Specificity  F1 Score  
0             87     0.256071     0.751941  0.412413  


In [11]:
# resnet152 model training with MR1_3
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 128.1542932987213
Model saved to comparison/models/resnet152_MR1_3_class.pth


In [12]:
# adding resnet152 MR1_3 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  resnet152  MR1_3  0.233333       0.25  1.071429   

   True positive  Sensitivity  Specificity  F1 Score  
0             84         0.25         0.75  0.405405  


In [13]:
# resnet152 model training with MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 123.50931084156036
Model saved to comparison/models/resnet152_MR1_4_class.pth


In [14]:
# adding resnet152 MR1_4 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  resnet152  MR1_4  0.220065       0.25  1.136029   

   True positive  Sensitivity  Specificity  F1 Score  
0             68         0.25         0.75  0.409814  


##ALexNet

In [15]:
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
alexnet = models.alexnet(pretrained=True)
model_name = 'alexnet'
# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel, self).__init__()
        self.vit_model = vit_model
        self.alexnet = models.alexnet(pretrained=True)
        # for param in self.alexnet.parameters():
        #     param.requires_grad = False
        self.linear_layer1 = nn.Linear(1000, 100)
        # self.linear_layer1.weight.data.normal_(mean=0.0, std=0.02)
        self.linear_layer2 = nn.Linear(100, 4)
        # self.linear_layer2.weight.data.normal_(mean=0.0, std=0.02)
        self.relu = nn.ReLU()
        self.out_1 = nn.Linear(9, 4)
        # self.out_1.weight.data.normal_(mean=0.0, std=0.02)
        self.out_2 = nn.Linear(5, 4)
        # self.out_2.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, inputs, x):
        # Preprocess the input tensor
        inputs = inputs.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=inputs, return_tensors="pt")
        outputs = self.alexnet(inputs['pixel_values'])
        outputs = self.linear_layer1(outputs)
        outputs = self.relu(outputs)
        final_output = self.linear_layer2(outputs)
        final_output = torch.cat((final_output, x), 1)
        final_output = self.relu(final_output)
        final_output = self.out_1(final_output)
        # final_output = self.relu(final_output)
        # final_output = self.out_2(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel(vit_model).to(device)
# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

Device: cpu


In [16]:
# alexnet model training with MR1_1
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 144.70904684066772
Model saved to comparison/models/alexnet_MR1_1_class.pth


In [17]:
# adding alexnet MR1_1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type    model   mode  Accuracy  Precision   Recall  \
0  Classification  alexnet  MR1_1  0.281337    0.25631  0.91131   

   True positive  Sensitivity  Specificity  F1 Score  
0            101      0.25631     0.752145  0.400092  


In [18]:
# alexnet model training with MR1_2
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 159.45099675655365
Model saved to comparison/models/alexnet_MR1_2_class.pth


In [19]:
# adding alexnet MR1_2 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type    model   mode  Accuracy  Precision  Recall  True positive  \
0  Classification  alexnet  MR1_2  0.278552       0.25  0.8975            100   

   Sensitivity  Specificity  F1 Score  
0         0.25         0.75  0.391068  


In [20]:
# alexnet model training with MR1_3
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 131.57871890068054
Model saved to comparison/models/alexnet_MR1_3_class.pth


In [21]:
# adding alexnet MR1_3 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type    model   mode  Accuracy  Precision  Recall  True positive  \
0  Classification  alexnet  MR1_3  0.286111   0.255677  0.8923            103   

   Sensitivity  Specificity  F1 Score  
0     0.255677     0.751995  0.397465  


In [22]:
# alexnet model training with MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 125.85784435272217
Model saved to comparison/models/alexnet_MR1_4_class.pth


In [23]:
# adding alexnet MR1_4 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type    model   mode  Accuracy  Precision    Recall  \
0  Classification  alexnet  MR1_4  0.281553       0.25  0.887931   

   True positive  Sensitivity  Specificity  F1 Score  
0             87         0.25         0.75  0.390152  


## VGG16

In [24]:
# vgg16 model
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
model_name = 'vgg16'
# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel, self).__init__()
        self.vit_model = vit_model
        self.vgg16 = models.vgg16(pretrained=True)
        # for param in self.vgg16.parameters():
        #     param.requires_grad = False
        self.linear_layer1 = nn.Linear(1000, 100)
        # self.linear_layer1.weight.data.normal_(mean=0.0, std=0.02)
        self.linear_layer2 = nn.Linear(100, 4)
        # self.linear_layer2.weight.data.normal_(mean=0.0, std=0.02)
        self.relu = nn.ReLU()
        self.out_1 = nn.Linear(9, 4)
        # self.out_1.weight.data.normal_(mean=0.0, std=0.02)
        self.out_2 = nn.Linear(5, 4)
        # self.out_2.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, inputs, x):
        # Preprocess the input tensor
        inputs = inputs.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=inputs, return_tensors="pt")
        outputs = self.vgg16(inputs['pixel_values'])
        outputs = self.linear_layer1(outputs)
        outputs = self.relu(outputs)
        final_output = self.linear_layer2(outputs)
        final_output = torch.cat((final_output, x), 1)
        final_output = self.relu(final_output)
        final_output = self.out_1(final_output)
        # final_output = self.relu(final_output)
        # final_output = self.out_2(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel(vit_model).to(device)
# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

Device: cpu


In [25]:
# vgg16 model training with MR1_1
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 146.53814339637756
Model saved to comparison/models/vgg16_MR1_1_class.pth


In [26]:
# adding vgg16 MR1_1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type  model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification  vgg16  MR1_1  0.259053   0.250721  0.966857             93   

   Sensitivity  Specificity  F1 Score  
0     0.250721     0.750112  0.398186  


In [27]:
# vgg16 model training with MR1_2
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 168.13059902191162
Model saved to comparison/models/vgg16_MR1_2_class.pth


In [28]:
# adding vgg16 MR1_2 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type  model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification  vgg16  MR1_2  0.259053       0.25  0.965054             93   

   Sensitivity  Specificity  F1 Score  
0         0.25         0.75  0.397124  


In [29]:
# vgg16 model training with MR1_3
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 129.16510939598083
Model saved to comparison/models/vgg16_MR1_3_class.pth


In [30]:
# adding vgg16 MR1_3 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type  model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification  vgg16  MR1_3  0.233333   0.231207  1.000065             84   

   Sensitivity  Specificity  F1 Score  
0     0.231207     0.744477  0.375583  


In [31]:
# vgg16 model training with MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 126.02128326892853
Model saved to comparison/models/vgg16_MR1_4_class.pth


In [32]:
# adding vgg16 MR1_4 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type  model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification  vgg16  MR1_4  0.268608   0.256436  0.957741             83   

   Sensitivity  Specificity  F1 Score  
0     0.256436     0.752449  0.404553  


GoogleNet

In [33]:
# googlenet model
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
model_name = 'googlenet'
# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel, self).__init__()
        self.vit_model = vit_model
        self.googlenet = models.googlenet(pretrained=True)
        # for param in self.googlenet.parameters():
        #     param.requires_grad = False
        self.linear_layer1 = nn.Linear(1000, 100)
        # self.linear_layer1.weight.data.normal_(mean=0.0, std=0.02)
        self.linear_layer2 = nn.Linear(100, 4)
        # self.linear_layer2.weight.data.normal_(mean=0.0, std=0.02)
        self.relu = nn.ReLU()
        self.out_1 = nn.Linear(9, 4)
        # self.out_1.weight.data.normal_(mean=0.0, std=0.02)
        self.out_2 = nn.Linear(5, 4)
        # self.out_2.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, inputs, x):
        # Preprocess the input tensor
        inputs = inputs.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=inputs, return_tensors="pt")
        outputs = self.googlenet(inputs['pixel_values'])
        outputs = self.linear_layer1(outputs)
        outputs = self.relu(outputs)
        final_output = self.linear_layer2(outputs)
        final_output = torch.cat((final_output, x), 1)
        final_output = self.relu(final_output)
        final_output = self.out_1(final_output)
        # final_output = self.relu(final_output)
        # final_output = self.out_2(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel(vit_model).to(device)
# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

Device: cpu


In [34]:
# googlenet model training with MR1_1
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 140.28503584861755
Model saved to comparison/models/googlenet_MR1_1_class.pth


In [35]:
# adding googlenet MR1_1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  googlenet  MR1_1  0.233983       0.25  1.068452   

   True positive  Sensitivity  Specificity  F1 Score  
0             84         0.25         0.75  0.405192  


In [36]:
# googlenet model training with MR1_2
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 163.30921471118927
Model saved to comparison/models/googlenet_MR1_2_class.pth


In [37]:
# adding googlenet MR1_2 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  googlenet  MR1_2  0.233983       0.25  1.068452   

   True positive  Sensitivity  Specificity  F1 Score  
0             84         0.25         0.75  0.405192  


In [38]:
# googlenet model training with MR1_3
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 127.98219585418701
Model saved to comparison/models/googlenet_MR1_3_class.pth


In [39]:
# adding googlenet MR1_3 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  googlenet  MR1_3  0.233333       0.25  1.071429   

   True positive  Sensitivity  Specificity  F1 Score  
0             84         0.25         0.75  0.405405  


In [40]:
# googlenet model training with MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(epochs):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 123.73360657691956
Model saved to comparison/models/googlenet_MR1_4_class.pth


In [41]:
# adding googlenet MR1_4 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type      model   mode  Accuracy  Precision    Recall  \
0  Classification  googlenet  MR1_4  0.216828   0.246324  1.134774   

   True positive  Sensitivity  Specificity  F1 Score  
0             67     0.246324     0.748835  0.404782  


Vision Transformers

In [8]:
# vision transformer model
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
model_name = 'vit'
# Freeze the ViT model parameters
for param in vit_model.parameters():
    param.requires_grad = False

class CombinedModel(nn.Module):
    def __init__(self, vit_model):
        super(CombinedModel, self).__init__()
        self.vit_model = vit_model
        for param in self.vit_model.parameters():
            param.requires_grad = False
        # self.googlenet = models.googlenet(pretrained=True)
        # for param in self.googlenet.parameters():
        #     param.requires_grad = False
        self.linear_layer1 = nn.Linear(self.vit_model.config.hidden_size, 100)
        # self.linear_layer1.weight.data.normal_(mean=0.0, std=0.02)
        self.linear_layer2 = nn.Linear(100, 4)
        # self.linear_layer2.weight.data.normal_(mean=0.0, std=0.02)
        self.relu = nn.ReLU()
        self.out_1 = nn.Linear(9, 4)
        # self.out_1.weight.data.normal_(mean=0.0, std=0.02)
        self.out_2 = nn.Linear(5, 4)
        # self.out_2.weight.data.normal_(mean=0.0, std=0.02)

    def forward(self, inputs, x):
        # Preprocess the input tensor
        inputs = inputs.mean(dim=0, keepdim=True).repeat(3, 1, 1)
        inputs = feature_extractor(images=inputs, return_tensors="pt")
        outputs = self.vit_model(inputs['pixel_values'])
        

        outputs = outputs.last_hidden_state
        outputs = outputs.mean(dim=1)
        outputs = self.linear_layer1(outputs)


        outputs = self.relu(outputs)
        final_output = self.linear_layer2(outputs)
        final_output = torch.cat((final_output, x), 1)
        final_output = self.relu(final_output)
        final_output = self.out_1(final_output)
        # final_output = self.relu(final_output)
        # final_output = self.out_2(final_output)
        return final_output

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
model = CombinedModel(vit_model).to(device)
# Define the loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
epochs_vit = epochs*10

Device: cpu


In [53]:

# vit model training with MR1_1
lr = 1e-6

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_1'

for epoch in range(epochs_vit):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num*2):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 292.01877522468567
Epoch 100 loss: 275.2869453430176
Epoch 200 loss: 266.5556879043579
Epoch 300 loss: 280.24893391132355
Epoch 400 loss: 247.8764853477478
Epoch 500 loss: 298.5896394252777
Epoch 600 loss: 267.8585307598114
Epoch 700 loss: 278.7196400165558
Epoch 800 loss: 271.32297027111053
Epoch 900 loss: 271.2293030023575
Model saved to comparison/models/vit_MR1_1_class.pth


In [54]:
# adding vit MR1_1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification   vit  MR1_1  0.259053       0.25  0.965054             93   

   Sensitivity  Specificity  F1 Score  
0         0.25         0.75  0.397124  


In [55]:
# vit model training with MR1_2
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_2'

for epoch in range(epochs_vit):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num*2):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 299.6700031757355
Epoch 100 loss: 262.99399280548096
Epoch 200 loss: 294.5311179161072
Epoch 300 loss: 292.05012118816376
Epoch 400 loss: 291.390473484993
Epoch 500 loss: 297.6241191625595
Epoch 600 loss: 286.49306654930115
Epoch 700 loss: 280.6008310317993
Epoch 800 loss: 289.60587751865387
Epoch 900 loss: 319.149694442749
Model saved to comparison/models/vit_MR1_2_class.pth


In [56]:
# adding vit MR1_2 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification   vit  MR1_2  0.259053       0.25  0.965054             93   

   Sensitivity  Specificity  F1 Score  
0         0.25         0.75  0.397124  


In [57]:
# vit model training with MR1_3
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_3'

for epoch in range(epochs_vit):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num*2):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 272.4692431688309
Epoch 100 loss: 279.3672202825546
Epoch 200 loss: 307.64523482322693
Epoch 300 loss: 303.6282926797867
Epoch 400 loss: 316.57406306266785
Epoch 500 loss: 252.22966754436493
Epoch 600 loss: 301.965155005455
Epoch 700 loss: 297.5401140451431


KeyboardInterrupt: 

In [48]:
# adding vit MR1_3 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification   vit  MR1_3  0.255556       0.25  0.978261             92   

   Sensitivity  Specificity  F1 Score  
0         0.25         0.75   0.39823  


In [49]:
# vit model training with MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1_4'

for epoch in range(epochs_vit):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num*2):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        if mode in filename:
            data = np.load(index[random_index][0])
            data = data/255
            label = index[random_index][1]
            code = index[random_index][0][-14:-10]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            optimizer.zero_grad()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

Epoch 0 loss: 249.9323261976242
Epoch 100 loss: 295.1977998018265
Model saved to comparison/models/vit_MR1_4_class.pth


In [50]:
# adding vit MR1_4 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type model   mode  Accuracy  Precision    Recall  True positive  \
0  Classification   vit  MR1_4  0.265372       0.25  0.942073             82   

   Sensitivity  Specificity  F1 Score  
0         0.25         0.75  0.395141  


In [51]:
classification_dataframe.to_csv('comparison/test/classification_results.csv', index=False)

In [13]:
# MR1_1, MR1_2, MR1_3, MR1_4
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

model = CombinedModel(vit_model).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

mode_list = ['MR1_1', 'MR1_2', 'MR1_3', 'MR1_4']
mode = 'MR1'

for epoch in range(epochs_vit):
    running_loss = 0.0
    results = [['code', 'Predicted class', 'True class']]
    for i in range(num*2):
        random_index = random.randint(0, len(index)-1)
        filename = index[random_index][0]
        data = np.load(index[random_index][0])
        data = data/255
        label = index[random_index][1]
        code = index[random_index][0][-14:-10]
        x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
        x[:, 0] = (x[:, 0] - 18) / (96 - 18)
        x[:, 1] = (x[:, 1] - 5) / (5 - 1)
        x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
        x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
        x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
        if code not in patients_codes_set:
            x = np.zeros((1, 5))
        x = torch.tensor(x).float()
        y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
        model.train()
        optimizer.zero_grad()
        input_tensor = torch.clamp(torch.tensor(data), 0, 1)
        outputs = model(input_tensor, x)
        target = torch.tensor([label]).to(device)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        results.append((code, np.argmax(outputs.detach().numpy()), label))
    if epoch % 100 ==0:
        print(f'Epoch {epoch} loss: {running_loss}')

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/results/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

model_save_path = 'comparison/models/'+model_name+'_'+mode+'_'+'class.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

model_test = CombinedModel(vit_model)

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Epoch 0 loss: 1113.9767614603043
Epoch 100 loss: 1113.8881492614746
Epoch 200 loss: 1112.9629156589508
Epoch 300 loss: 1112.7155607938766
Epoch 400 loss: 1117.6217091083527
Epoch 500 loss: 1111.4327956438065
Epoch 600 loss: 1107.3745658397675
Epoch 700 loss: 1110.2739493846893
Epoch 800 loss: 1109.8170759677887
Epoch 900 loss: 1107.6408507823944
Model saved to comparison/models/vit_MR1_class.pth


In [14]:
# adding vit MR1 results to the dataframe
running_loss = 0.0
# results = []
results = [['code', 'Predicted class', 'True class']]
for folder in os.listdir(test_folder):
    if os.path.isdir(test_folder + '/'+folder):
        folder_path = test_folder+'/'+folder
        files = [f for f in os.listdir(folder_path) if f.endswith('.npy') and mode in f]
        if folder == 'NonDemented':
            label = 0
        elif folder == 'VeryMildDemented':
            label = 1
        elif folder == 'MildDemented':
            label = 2
        elif folder == 'ModerateDemented':
            label = 3
        for f in files :
            data = np.load(os.path.join(folder_path, f))
            code = f[0:4]
            x = data_frame[data_frame['ID'].str.contains(code)][['Age', 'Educ', 'eTIV', 'nWBV', 'ASF']].to_numpy()
            x[:, 0] = (x[:, 0] - 18) / (96 - 18)
            x[:, 1] = (x[:, 1] - 5) / (5 - 1)
            x[:, 2] = (x[:, 2] - 1123) / (1992 - 1123)
            x[:, 3] = (x[:, 3] - 0.644) / (0.893 - 0.644)
            x[:, 4] = (x[:, 4] - 0.881) / (1.563 - 0.881)
            if code not in patients_codes_set:
                x = np.zeros((1, 5))
            x = torch.tensor(x).float()
            y = data_frame[data_frame['ID'].str.contains(code)][['MMSE', 'CDR', 'SES']].to_numpy()
            model.train()
            input_tensor = torch.clamp(torch.tensor(data), 0, 1)
            outputs = model(input_tensor, x)
            target = torch.tensor([label]).to(device)
            loss = criterion(outputs, target)
            running_loss += loss.item()
            results.append((code, np.argmax(outputs.detach().numpy()), label))

results = pd.DataFrame(results[1:],columns=results[0])
results.to_csv('comparison/test/'+model_name+'_'+mode+'_'+'results_class.csv', index=False)

accuracy = (results['Predicted class'] == results['True class']).sum() / len(results)
# print(f'Accuracy: {accuracy}')

pre_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
pre_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
pre_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
pre_3 = +((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
pre_list = [pre_0, pre_1, pre_2, pre_3]
pre_list = [ele for ele in pre_list if not math.isnan(ele)]
precision = sum(pre_list) / len(pre_list)
# print(f'Precision: {precision}')

recall_0 = (results['Predicted class'] == 0).sum() / ((results['True class'] == 0).sum())
recall_1 = (results['Predicted class'] == 1).sum() / ((results['True class'] == 1).sum())
recall_2 = (results['Predicted class'] == 2).sum() / ((results['True class'] == 2).sum())
recall_3 = (results['Predicted class'] == 3).sum() / ((results['True class'] == 3).sum())
recall_list = [recall_0, recall_1, recall_2, recall_3]
recall_list = [ele for ele in recall_list if not math.isnan(ele)]
recall = sum(recall_list) / len(recall_list)
# print(f'Recall: {recall}')

true_positive = (results['Predicted class'] == results['True class']).sum()
# print(f'True positive: {true_positive}')

sen_0 = ((results['Predicted class'] == 0) & (results['True class'] == 0)).sum() / ((results['True class'] == 0).sum())
sen_1 = ((results['Predicted class'] == 1) & (results['True class'] == 1)).sum() / ((results['True class'] == 1).sum())
sen_2 = ((results['Predicted class'] == 2) & (results['True class'] == 2)).sum() / ((results['True class'] == 2).sum())
sen_3 = ((results['Predicted class'] == 3) & (results['True class'] == 3)).sum() / ((results['True class'] == 3).sum())
sen_list = [sen_0, sen_1, sen_2, sen_3]
sen_list = [ele for ele in sen_list if not math.isnan(ele)]
sensitivity = sum(sen_list) / len(sen_list)
# print(f'Sensitivity: {sensitivity}')

spec_0 = ((results['Predicted class'] != 0) & (results['True class'] != 0)).sum() / (((results['Predicted class'] != 0) & (results['True class'] != 0)).sum()+((results['Predicted class'] == 0) & (results['True class'] != 0)).sum())
spec_1 = ((results['Predicted class'] != 1) & (results['True class'] != 1)).sum() / (((results['Predicted class'] != 1) & (results['True class'] != 1)).sum()+((results['Predicted class'] == 1) & (results['True class'] != 1)).sum())
spec_2 = ((results['Predicted class'] != 2) & (results['True class'] != 2)).sum() / (((results['Predicted class'] != 2) & (results['True class'] != 2)).sum()+((results['Predicted class'] == 2) & (results['True class'] != 2)).sum())
spec_3 = ((results['Predicted class'] != 3) & (results['True class'] != 3)).sum() / (((results['Predicted class'] != 3) & (results['True class'] != 3)).sum()+ ((results['Predicted class'] == 3) & (results['True class'] != 3)).sum())
specificity_list = [spec_0, spec_1, spec_2, spec_3]
specificity_list = [ele for ele in specificity_list if not math.isnan(ele)]
specificity = sum(specificity_list) / len(specificity_list)
# print(f'Specificity: {specificity}')

f1_score = 2 * (precision * recall) / (precision + recall)
# print(f'F1 Score: {f1_score}')

new_data = pd.DataFrame([{'Type': 'Classification', 'model': model_name, 'mode': mode, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'True positive': true_positive, 'Sensitivity': sensitivity, 'Specificity': specificity, 'F1 Score': f1_score}])

classification_dataframe = pd.concat([classification_dataframe, new_data], ignore_index=True)

print(new_data)

             Type model mode  Accuracy  Precision    Recall  True positive  \
0  Classification   vit  MR1  0.259553       0.25  0.963194            360   

   Sensitivity  Specificity  F1 Score  
0         0.25         0.75  0.396966  
